In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import random
from googletrans import Translator

In [59]:
med = Translator().translate('Pembrolizumab', src='es', dest='en').text.replace(' ', '+')
med

'pembrolizumab'

In [60]:
url='https://www.pharmacychecker.com/'+med+'/?src=drug-suggest#prices'
url

'https://www.pharmacychecker.com/pembrolizumab/?src=drug-suggest#prices'

In [61]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')

In [62]:
driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"),options=options)
driver.get(url)
#time.sleep(random.randint(5,10))
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')
driver.quit()

In [63]:
results=soup.find_all("li",{"class":"drug-pricing-item mt-2"})

In [64]:
if not results:
    df=pd.DataFrame()  
else:
    data=[]
    for result in results:
        info={}
        info["precio"]=result.find("p",{"class":"desktop-price"}).text
        info["farmacia"] = result.find("a", {"class": "cpc-drug-listings-link-image"}).get("data-pharm")
        info["producto"] = result.find("a", {"class": "cpc-drug-listings-link-image"}).get("data-drug")
        info["cantidad"] = result.find("div", {"class": "col-3 col-lg-3 drug-pricing-details hide-on-mobile"}).text
        #Añadir país
        data.append(info)
    df=pd.DataFrame(data)
    #Añadir pais
    canada_pharmacies = ['liferxpharmacy', 'canadianprescriptiondrugstore', 'medsengage', 'offshorecheapmeds', 'pricepropharmacy']
    df["pais"] = df["farmacia"].apply(lambda x: "Canada" if any([pharmacy in x for pharmacy in canada_pharmacies]) else "Estados Unidos")

In [65]:
df

""
